In [1]:
!date

Wed Feb 16 09:55:05 CET 2022


In [2]:
import fixed2float
from fixed2float import Fx, to_Fx

fixed2float.__version__

'2.0.1'

In [3]:
N = 8
M = 9
M2 = 12

In [4]:
# let's say we have this posit

from hardposit import from_bits

a = from_bits(0b0111000000101000, 16, 1)
a

P<16,1>:         0b0111000000101000   0x7028   28712
                   0111000000101000   
sign:              0
regime:            0000000000001110 -> (reg_s, reg_len) = (1, 4) -> k = 2
exponent:                         0
mantissa:          0000000000101000 -> 1000010100000000 (1.0390625)
                   0111000000101000

                   (-1) ** 0 * (2 ** (2 ** 1)) ** 2 * (2 ** 0) * (1 + 40/1024)
                   16.625
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [5]:
fp_mant_a = Fx(a.mant_repr().unwrap(), 1, 16)
fp_mant_a

1000010100000000

In [6]:
# now we trim the mantissa to N bits and remove the "1." which provides to additional information

fp_frac_a_N_bits = to_Fx(
    x=fp_mant_a.eval() - 1,
    m=0,
    b=N,
)

fp_frac_a_N_bits

00001010

In [7]:
type(_)

Fx

In [8]:
# so the net value that enters the LUT is:

fp_frac_a_N_bits.val

10

In [9]:
fp_mant_a, fp_mant_a.eval()

(1000010100000000, 1.0390625)

In [10]:
fp_mant_a_recip1 = to_Fx(1.0 / fp_mant_a.eval(), 1, 1 + M)
fp_mant_a_recip2 = to_Fx(1.0 / fp_mant_a.eval(), 1, 1 + M2)


fp_mant_a_recip1, fp_mant_a_recip2

(0111101101..., 0111101100110...)

In [11]:
# most precise vs (1+M)-bits precise

1.0 / fp_mant_a.eval(), fp_mant_a_recip1.eval(), fp_mant_a_recip2.eval()

(0.9624060150375939, 0.962890625, 0.96240234375)

In [12]:
to_Fx(fp_mant_a_recip1.eval(), 0, M), to_Fx(fp_mant_a_recip2.eval(), 0, M2)

(111101101, 111101100110)

In [13]:
_[0].val, _[1].val

(493, 3942)

In [14]:
hex(_[0]), hex(_[1])

('0x1ed', '0xf66')

In [15]:
# that is ultimately the _value_ corresponding to the _key_ 10

#             8'd10 :    dout <= 9'h1ed;